In [1]:
class Reinforcement:
    # Defautl Goal_steps , LR, score_requirement and initia_games Defined  
    def __init__( self,goal_steps = 500,LR = 1e-3,score_requirement = 100,initial_games = 10000):
        print("Unsupervised Instance Begin") 
        print("__init__( self): Always executed when the class is being initiated")
        self.LR = LR
        self.goal_steps = goal_steps
        self.score_requirement = score_requirement
        self.initial_games = initial_games 

        
    def __del__(self):
        print("Unsupervised Instance Destroyed")
    
    def check(self):
        print("In Unsupervised class")
        
     # 1.1
    # Pole Balance
    # Reinforcement 
    # Make it Better
    def Pole_Balance(self):
        import gym
        import random
        import numpy as np
        import tflearn
        from tflearn.layers.core import input_data, dropout, fully_connected
        from tflearn.layers.estimator import regression
        from statistics import median, mean
        from collections import Counter

        LR = 1e-3
        env = gym.make("CartPole-v0")
        env.reset()
        goal_steps = 500
        score_requirement = 100
        initial_games = 10000
        return score_requirement

    def some_random_games_first(self):
        # Each of these is its own game.
        for episode in range(5):
            env.reset()
            # this is each frame, up to 200...but we wont make it that far.
            for t in range(200):
                # This will display the environment
                # Only display if you really want to see it.
                # Takes much longer to display it.
                env.render()

                # This will just create a sample action in any environment.
  
# In this environment, the action can be 0 or 1, which is left or right
                action = env.action_space.sample()

                # this executes the environment with an action, 
                # and returns the observation of the environment, 
                # the reward, if the env is over, and other info.
                observation, reward, done, info = env.step(action)
                if done:
                    break

    #some_random_games_first()
    def initial_population(self):
        import numpy as np
        import random
        import gym
        from statistics import median, mean
        from collections import Counter
        
        env = gym.make("CartPole-v0")
        env.reset()
        
        # [OBS, MOVES]
        training_data = []
        # all scores:
        scores = []
        # just the scores that met our threshold:
        accepted_scores = []
        # iterate through however many games we want:
        for _ in range(self.initial_games):
            score = 0
            # moves specifically from this environment:
            game_memory = []
            # previous observation that we saw
            prev_observation = []
            # for each frame in 200
            for _ in range(self.goal_steps):
                # choose random action (0 or 1)
                action = random.randrange(0,2)
                # do it!
                observation, reward, done, info = env.step(action)

                # notice that the observation is returned FROM the action
                # so we'll store the previous observation here, pairing
                # the prev observation to the action we'll take.
                if len(prev_observation) > 0 :
                    game_memory.append([prev_observation, action])
                prev_observation = observation
                score+=reward
                if done: break

            # IF our score is higher than our threshold, we'd like to save
            # every move we made
            # NOTE the reinforcement methodology here. 
            # all we're doing is reinforcing the score, we're not trying 
            # to influence the machine in any way as to HOW that score is 
            # reached.
            if score >= self.score_requirement:
                accepted_scores.append(score)
                for data in game_memory:
                    # convert to one-hot (this is the output layer for our neural network)
                    if data[1] == 1:
                        output = [0,1]
                    elif data[1] == 0:
                        output = [1,0]

                    # saving our training data
                    training_data.append([data[0], output])

            # reset env to play again
            env.reset()
            # save overall scores
            scores.append(score)

        # just in case you wanted to reference later
        training_data_save = np.array(training_data)
        np.save('saved1.npy',training_data_save)

        # some stats here, to further illustrate the neural network magic!
        print('Average accepted score:',mean(accepted_scores))
        print('Median score for accepted scores:',median(accepted_scores))
        print(Counter(accepted_scores))

        return training_data



    def neural_network_model(self, input_size):
        import tflearn
        from tflearn.layers.core import input_data, dropout, fully_connected
        from tflearn.layers.estimator import regression

        network = input_data(shape=[None, input_size, 1], name='input')

        network = fully_connected(network, 128, activation='relu')
        network = dropout(network, 0.8)

        network = fully_connected(network, 256, activation='relu')
        network = dropout(network, 0.8)

        network = fully_connected(network, 512, activation='relu')
        network = dropout(network, 0.8)

        network = fully_connected(network, 512, activation='relu')
        network = dropout(network, 0.8)

        network = fully_connected(network, 512, activation='relu')
        network = dropout(network, 0.8)

        network = fully_connected(network, 2, activation='softmax')
        network = regression(network, optimizer='adam', learning_rate=self.LR, loss='categorical_crossentropy', name='targets')
        model = tflearn.DNN(network, tensorboard_dir='log')

        return model

    def train_model(self,training_data, model=False):
        import numpy as np

        X = np.array([i[0] for i in training_data]).reshape(-1,len(training_data[0][0]),1)
        y = [i[1] for i in training_data]

        if not model:
            model = Reinforcement.neural_network_model(self,input_size = len(X[0]))

        model.fit({'input': X}, {'targets': y}, n_epoch=3, snapshot_step=500, show_metric=True, run_id='openai_learning')
        return model

    # model = train_model(initial_population())

    def Pole_test(self):
        # 1.2
        import gym
        import random
        import numpy as np
        import tflearn
        from tflearn.layers.core import input_data, dropout, fully_connected
        from tflearn.layers.estimator import regression
        from statistics import median, mean
        from collections import Counter

        LR = 1e-3
        env = gym.make("CartPole-v0")
        env.reset()
        goal_steps = 500
        score_requirement = 50
        initial_games = 10000
        training_data =initial_population()
        model = train_model(training_data)
        scores = []
        choices = []

        for each_game in range(10):
            score = 0
            game_memory = []
            prev_obs = []
            env.reset()
            for _ in range(goal_steps):
                env.render()
                if len(prev_obs)==0:
                    action = random.randrange(0,2)
                else:
                    action = np.argmax(model.predict(prev_obs.reshape(-1,len(prev_obs),1))[0])
                choices.append(action)               
                new_observation, reward, done, info = env.step(action)
                prev_obs = new_observation
                game_memory.append([new_observation, action])
                score+=reward
                if done: break
            scores.append(score)

        print('Average Score:',sum(scores)/len(scores))
        print('choice 1:{}  choice 0:{}'.format(choices.count(1)/len(choices),choices.count(0)/len(choices)))
        print(score_requirement)


In [2]:
r=Reinforcement()
 

Unsupervised Instance Begin
__init__( self): Always executed when the class is being initiated


In [142]:
r.Pole_Balance()
traindata=r.goal_steps
print(r.LR)

WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
0.001


In [3]:
r.train_model(r.initial_population())

Training Step: 14  | total loss: 0.69264 | time: 0.137s
| Adam | epoch: 003 | loss: 0.69264 - acc: 0.5348 -- iter: 256/315
Training Step: 15  | total loss: 0.69293 | time: 0.165s
| Adam | epoch: 003 | loss: 0.69293 - acc: 0.5151 -- iter: 315/315
--
